In [ ]:
pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 14.2 MB/s eta 0:00:00


In [ ]:
 !pip install itk-tubetk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 63.8 MB/s eta 0:00:00


In [1]:
import numpy as np
import SimpleITK as sitk
import sys
import itk
from itk import TubeTK as ttk



In [51]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib.pyplot as plt


Segment brain from MRA

In [9]:
mra = '...\sub-013_ses-20111130_desc-angio_N4bfc_mask.nii.gz'
mri_T1='...\sub-013_ses-20111130_T1w.nii.gz'
flash='...\sub-013_ses-20111130_angio.nii.gz'
flash_Brain='...\sub-013_ses-20111130_desc-angio_N4bfc_brain_mask.nii.gz'
# Specify the path to the output MetaImage file
mha_mra = '...\sub_013\\sub-130_mra.mha'
mha_mri_T1 ='...\sub-013_mri_T1.mha'
mha_flash='...\sub-013_flash.mha'
mha_flash_Brain='...\sub-013_flash-Brain.mha'


# Read the input NIfTI file
image_mra= sitk.ReadImage(mra)
image_mri_T1=sitk.ReadImage(mri_T1)
image_flash=sitk.ReadImage(flash)
image_flash_Brain=sitk.ReadImage(flash_Brain)

# Write the output MHA file
sitk.WriteImage(image_mra, mha_mra)
sitk.WriteImage(image_mri_T1,mha_mri_T1)
sitk.WriteImage(image_flash,mha_flash)
sitk.WriteImage(image_flash_Brain,mha_flash_Brain)

In [10]:
ImageType = itk.Image[itk.F, 3]
ReaderType = itk.ImageFileReader[ImageType]

reader1 = ReaderType.New(FileName=mha_mra)
reader1.Update()
im1 = reader1.GetOutput()

reader2 = ReaderType.New(FileName=mri_T1)
reader2.Update()
im2 = reader2.GetOutput()

In [11]:
res = ttk.ResampleImage.New(Input = im1) 
res.SetMakeHighResIso(True)
res.Update()
im1iso = res.GetOutput()


In [12]:
imReg = ttk.RegisterImages[ImageType].New() 
imReg.SetFixedImage(im1iso)
imReg.SetMovingImage(im2)
imReg.SetReportProgress(True)
imReg.SetExpectedOffsetMagnitude(40)
imReg.SetExpectedRotationMagnitude(0.01)
imReg.SetExpectedScaleMagnitude(0.01)
imReg.SetRigidMaxIterations(500)
imReg.SetRigidSamplingRatio(0.1)
imReg.SetRegistration("RIGID")
imReg.SetMetric("MATTES_MI_METRIC")

In [13]:
imReg.Update()

In [14]:
im2iso = imReg.GetFinalMovingImage()

In [15]:
patient_flash = "...\sub-013_flash.mha"
patient_flash_brain = "...\sub-013_flash-Brain.mha"
reader = ReaderType.New(FileName=patient_flash)
reader.Update()
imBase = reader.GetOutput()
reader = ReaderType.New(FileName=patient_flash_brain)
reader.Update()
imBaseB = reader.GetOutput()

In [16]:
imMath = ttk.ImageMath.New(Input=im2iso)
imMath.Blur(2)
im2isoBlur = imMath.GetOutput()

In [17]:
regB = []
regBB = []

In [18]:
imMath.SetInput(imBase)
imMath.Blur(2)
imBaseBlur = imMath.GetOutput()
regBTo1 = ttk.RegisterImages[ImageType].New(FixedImage=im2isoBlur, MovingImage=imBaseBlur)
regBTo1.SetReportProgress(True)
regBTo1.SetExpectedOffsetMagnitude(40)
regBTo1.SetExpectedRotationMagnitude(0.01)
regBTo1.SetExpectedScaleMagnitude(0.1)
regBTo1.SetRigidMaxIterations(500)
regBTo1.SetAffineMaxIterations(500)
regBTo1.SetRigidSamplingRatio(0.05)
regBTo1.SetAffineSamplingRatio(0.05)
regBTo1.SetInitialMethodEnum("INIT_WITH_IMAGE_CENTERS")
regBTo1.SetRegistration("PIPELINE_AFFINE")
regBTo1.SetMetric("MATTES_MI_METRIC")
regBTo1.Update()
img = regBTo1.ResampleImage("LINEAR", imBase)
regB.append( img )
img = regBTo1.ResampleImage("LINEAR", imBaseB)
regBB.append( img )

In [19]:
N=1
regBBT = []
for i in range(0,N):
    print("Type of regBB[i]:", type(regBB[i]))
    imMath = ttk.ImageMath[itk.Image[itk.F,3]].New(Input=regBB[i])
    imMath.Threshold(0,1,0,1)
    img = imMath.GetOutput()
    if i==0:
        print("Type of img:", type(img))
        imMathSum = ttk.ImageMath[itk.Image[itk.F,3]].New( img )
        imMathSum.AddImages( img, 1.0/N, 0 )
        sumBBT = imMathSum.GetOutput()
    else:
        print("Type of img:", type(sumBBT))
        imMathSum = ttk.ImageMath[itk.Image[itk.F,3]].New( sumBBT )
        imMathSum.AddImages( img, 1, 1.0/N )
        sumBBT = imMathSum.GetOutput()
        

Type of regBB[i]: <class 'itk.itkImagePython.itkImageF3'>
Type of img: <class 'itk.itkImagePython.itkImageF3'>


In [20]:
insideMath = ttk.ImageMath[itk.Image[itk.F,3]].New( Input = sumBBT )
insideMath.Threshold(1,1,1,0)
insideMath.Erode(5,1,0)
brainInside = insideMath.GetOutput()

outsideMath = ttk.ImageMath[itk.Image[itk.F,3]].New( Input = sumBBT )
outsideMath.Threshold(0,0,1,0)
outsideMath.Erode(10,1,0)
brainOutsideAll = outsideMath.GetOutput()
outsideMath.Erode(20,1,0)
outsideMath.AddImages(brainOutsideAll, -1, 1)
brainOutside = outsideMath.GetOutput()

outsideMath.AddImages(brainInside,1,2)
brainCombinedMask = outsideMath.GetOutputUChar()

outsideMath.AddImages(im2iso, 512, 1)
brainCombinedMaskView = outsideMath.GetOutput()

In [21]:
LabelMapType = itk.Image[itk.UC,3]

segmenter = ttk.SegmentConnectedComponentsUsingParzenPDFs[ImageType,LabelMapType].New()
segmenter.SetFeatureImage( im1iso )
segmenter.AddFeatureImage( im2iso )
#segmenter.AddFeatureImage( im3iso )
segmenter.SetInputLabelMap( brainCombinedMask )
segmenter.SetObjectId( 2 )
segmenter.AddObjectId( 1 )
segmenter.SetVoidId( 0 )
segmenter.SetErodeDilateRadius( 5 )
segmenter.Update()
segmenter.ClassifyImages()
brainCombinedMaskClassified = segmenter.GetOutputLabelMap()

In [22]:
cast = itk.CastImageFilter[LabelMapType, ImageType].New()
cast.SetInput(brainCombinedMaskClassified)
cast.Update()
brainMaskF = cast.GetOutput()

brainMath = ttk.ImageMath[ImageType].New(Input = brainMaskF)
brainMath.Threshold(2,2,1,0)
brainMath.Dilate(2,1,0)
brainMaskD = brainMath.GetOutput()
brainMath.SetInput( im1 )
brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0)
brain = brainMath.GetOutput()


In [23]:
itk.imwrite(brainMaskD, "...\sub-013_MRMask-Brain.mha")

itk.imwrite(im1iso, "...\sub-013_MRA-Iso.mha")
itk.imwrite(im2iso, "...\sub-013_MRT1-Iso.mha")

itk.imwrite(brain, "...\sub-013_MRA-Brain.mha")

brainMath.SetInput(im2iso)
brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0 )
itk.imwrite(brainMath.GetOutput(), "...\sub-013_MRT1-Brain.mha")

Compute Vessel Probability image from Brain MRAs

In [24]:
ImageType = itk.Image[itk.F, 3]
ReaderType = itk.ImageFileReader[ImageType]
ResampleType = ttk.ResampleImage[ImageType]

#IMPORTANT: This assumed the data has been brain-stripped and resampled into isotropic spacing!!!
reader = ReaderType.New(FileName="...\sub-013_MRA-Iso.mha")
reader.Update()
im1iso = reader.GetOutput()


#reader = ReaderType.New(FileName="MRT2-Iso.mha")
#reader.Update()
##im3iso = reader.GetOutput()
#reader = ReaderType.New(FileName="MRT2-Brain.mha")
#reader.Update()
#im3Brain = reader.GetOutput()

In [25]:
reader = ReaderType.New(FileName="...\sub-013_MRA-Brain.mha")
reader.Update()
im1Brain = reader.GetOutput()

reader = ReaderType.New(FileName="...\sub-013_MRT1-Iso.mha")
reader.Update()
im2iso = reader.GetOutput()
reader = ReaderType.New(FileName="...\sub-013_MRT1-Brain.mha")
reader.Update()
im2Brain = reader.GetOutput()

In [26]:
imMath = ttk.ImageMath[itk.Image[itk.F,3]].New()
imMath.SetInput(im1Brain)
imMath.Blur(1.0)
imBlur = imMath.GetOutput()
imBlurArray = itk.GetArrayViewFromImage(imBlur)

numSeeds = 10
seedCoverage = 20
seedCoord = np.zeros([numSeeds,3])
for i in range(numSeeds):
    seedCoord[i] = np.unravel_index(np.argmax(imBlurArray, axis=None), imBlurArray.shape)
    indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
    minX = max(indx[0]-seedCoverage,0)
    maxX = max(indx[0]+seedCoverage,imBlurArray.shape[0])
    minY = max(indx[1]-seedCoverage,0)
    maxY = max(indx[1]+seedCoverage,imBlurArray.shape[1])
    minZ = max(indx[2]-seedCoverage,0)
    maxZ = max(indx[2]+seedCoverage,imBlurArray.shape[2])
    imBlurArray[minX:maxX,minY:maxY,minZ:maxZ]=0
    indx.reverse()
    seedCoord[:][i] = im1Brain.TransformIndexToPhysicalPoint(indx)
print(seedCoord)

[[-28.984375   -5.1953125 -11.       ]
 [ 28.7109375  -4.375     -12.       ]
 [ 21.6015625  -2.1875    -24.5      ]
 [-25.9765625  11.7578125 -35.5      ]
 [ 25.9765625  -5.46875   -35.       ]
 [ 23.515625    6.5625    -35.       ]
 [ 23.515625   12.3046875 -34.5      ]
 [-28.4375     18.046875  -38.5      ]
 [ 39.921875   -7.65625   -34.5      ]
 [-16.6796875  20.5078125 -41.5      ]]


In [27]:
# Manually extract a few vessels to form an image-specific training set
vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.1)
vSeg.SetMinCurvature(0.001)
vSeg.SetRadiusInObjectSpace( 1 )
for i in range(numSeeds):
    print("**** Processing seed " + str(i) + " : " + str(seedCoord[i]))
    vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    
tubeMaskImage = vSeg.GetTubeMaskImage()

**** Processing seed 0 : [-28.984375   -5.1953125 -11.       ]
**** Processing seed 1 : [ 28.7109375  -4.375     -12.       ]
**** Processing seed 2 : [ 21.6015625  -2.1875    -24.5      ]
**** Processing seed 3 : [-25.9765625  11.7578125 -35.5      ]
**** Processing seed 4 : [ 25.9765625  -5.46875   -35.       ]
**** Processing seed 5 : [ 23.515625   6.5625   -35.      ]
**** Processing seed 6 : [ 23.515625   12.3046875 -34.5      ]
**** Processing seed 7 : [-28.4375    18.046875 -38.5     ]
**** Processing seed 8 : [ 39.921875  -7.65625  -34.5     ]
**** Processing seed 9 : [-16.6796875  20.5078125 -41.5      ]


In [28]:
LabelMapType = itk.Image[itk.UC,3]

trMask = ttk.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 3 )
trMask.SetObjectWidth( 1 )
trMask.SetNotObjectWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [29]:
enhancer = ttk.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.SetInput( im1iso )
enhancer.AddInput( im2iso )
#enhancer.AddInput( im3iso )
enhancer.SetLabelMap( fgMask )
enhancer.SetRidgeId( 255 )
enhancer.SetBackgroundId( 128 )
enhancer.SetUnknownId( 0 )
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetScales([0.3333,1,2.5])
enhancer.Update()
enhancer.ClassifyImages()

In [31]:
im1vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))
itk.imwrite( im1vess, "...\sub013_MRA-VesselEnhanced_patient.mha", compression=True)

brainMask = itk.imread( "...\sub-013_MRMask-Brain.mha", itk.F )
im1BrainVess = itk.MultiplyImageFilter(Input1 = im1vess, Input2=brainMask)
itk.imwrite( im1BrainVess, "...\sub013-MRA-Brain-VesselEnhanced_patient.mha", compression=True)

Segementing Vessels

In [32]:
ImageType = itk.Image[itk.F, 3]

im1iso = itk.imread("...\sub013_MRA-VesselEnhanced_patient.mha")
im1BrainVess = itk.imread("...\sub013-MRA-Brain-VesselEnhanced_patient.mha")

In [33]:
imMath = ttk.ImageMath.New(im1BrainVess)
imMath.MedianFilter(1)

In [34]:
imMath.Threshold(0.0000000001, 1, 1, 0)
im1VessMask = imMath.GetOutputShort()


In [35]:
ccSeg = ttk.SegmentConnectedComponents.New(im1VessMask)
ccSeg.SetMinimumVolume(4)

In [36]:
ccSeg.Update()
im1VessMaskCC = ccSeg.GetOutput()

In [37]:
imMathSS = ttk.ImageMath.New(im1VessMaskCC)
imMath.Threshold(1,1,0,1)
im1VessMaskInv = imMath.GetOutputFloat()

In [38]:
distFilter = itk.DanielssonDistanceMapImageFilter.New(im1VessMaskInv)
distFilter.Update()
im1SeedRadius = distFilter.GetOutput()

In [39]:
numSeeds = 20
vSeg = ttk.SegmentTubes.New(Input=im1iso)

In [40]:
vSeg.SetMinCurvature(0.0)
vSeg.SetMinRoundness(0.0)


In [41]:
vSeg.SetMinRidgeness(0.8)
vSeg.SetMinLevelness(0.0)
vSeg.SetRadiusInObjectSpace( 0.75 )


In [42]:
vSeg.SetBorderInIndexSpace(3)
vSeg.SetSeedMask( im1SeedRadius )
vSeg.SetSeedRadiusMask( im1SeedRadius )

In [43]:
vSeg.SetOptimizeRadius(False)
vSeg.SetUseSeedMaskAsProbabilities(True)


In [44]:
vSeg.SetSeedExtractionMinimumProbability(0.4)
#vSeg.SetSeedMaskMaximumNumberOfPoints( numSeeds )
vSeg.ProcessSeeds()

In [47]:
SOWriter = itk.SpatialObjectWriter[3].New(vSeg.GetTubeGroup())
SOWriter.SetFileName( "...\sub-013_MRA-Vessels_patient.tre" )
SOWriter.Update()